In [1]:
import sys
sys.path.append("/Users/hari/personal/anime-rag-recommender")

In [2]:
import json
with open("../data/anime.jsonl", "r") as f:
    lines = f.readlines()

print(len(lines))

data = []
for line in lines:
    data.append(json.loads(line))

print(data[0])

2488
{'id': 1, 'title': 'Cowboy Bebop', 'synopsis': "Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between

In [11]:
from ingestion.schema import AnimeDocument

docs = [AnimeDocument.model_validate(d) for d in data[:1000]]

In [12]:
import time
start = time.perf_counter()
from indexing.chunking import build_semantic_chunks
end = time.perf_counter()
print(f"Time taken: {end - start}")

Time taken: 0.26205954200122505


In [13]:
chunks = []
print(len(docs))
start = time.perf_counter()
for doc in docs:
    chunks.extend(build_semantic_chunks(doc))
end = time.perf_counter()
print(f"Time taken: {end - start}")
print(len(chunks))

1000
Time taken: 0.012807832972612232
3819


In [14]:
from indexing.embedding import embed_texts

In [15]:
vecs = embed_texts(chunks[:10])
print(len(vecs), len(vecs[0]))

[2026-01-15 00:51:58] [INFO] [indexing.status] Embedding 10 chunks with sentence-transformers/all-MiniLM-L6-v2
10 384


In [16]:
AnimeDocument.model_validate_json(lines[10])


AnimeDocument(id=20, title='Naruto', synopsis="Twelve years ago, a colossal demon fox terrorized the world. During the monster's attack on the Hidden Leaf Village, the Hokage—the village's leader and most powerful ninja—sacrifices himself to seal the beast inside a newborn, relieving civilization from destruction while dooming the baby to a lonely life.\n\nNow, after years of being shunned and bullied, Naruto Uzumaki pesters the village with elaborate pranks and vandalism. Despite these antics, he works hard to achieve his dream: to become the Hokage and earn the acknowledgement of those who have mistreated him for his entire life. Naruto joins Team 7, a ninja squad made up of two of his peers—prodigy Sasuke Uchiha and clever Sakura Haruno.\n\nUnder the aloof Kakashi Hatake's leadership, Team 7 takes on a series of difficult missions, forcing its members to grow in strength and comradery despite their many differences. Naruto strives to stand out in his rivalry with Sasuke and earn the

In [3]:
import chromadb
from chromadb.config import Settings


In [19]:
_CHROMA_DIR = "/Users/hari/personal/anime-rag-recommender/chroma_db"
_COLLECTION_NAME = "anime_chunks"

_client = chromadb.PersistentClient(
    path=_CHROMA_DIR,
    settings = Settings(
        anonymized_telemetry=False,
    )
)


_collection = _client.get_or_create_collection(_COLLECTION_NAME)

In [20]:
_collection.count()

8891

In [6]:
query = "adult sci fi anime in space"

In [7]:
from indexing.embedding import embed_texts

In [8]:
query_embedding = embed_texts([query])[0]
results = _collection.query(query_embeddings=[query_embedding])

[2026-01-15 00:52:32] [INFO] [indexing.status] Embedding 1 chunks with sentence-transformers/all-MiniLM-L6-v2


In [10]:
documents = results["documents"][0]
metadatas = results["metadatas"][0]
distances = results["distances"][0]

In [13]:
len(distances)

10

In [14]:
from retrieval.search import search

In [15]:
results = search(query)

[2026-01-15 00:53:40] [INFO] [retrieval.status] Searching for: adult sci fi anime in space
[2026-01-15 00:53:40] [INFO] [indexing.status] Embedding 1 chunks with sentence-transformers/all-MiniLM-L6-v2


In [17]:
from retrieval.search import search

results = search("adult sci fi anime in space")
for r in results:
    print(r)

[2026-01-15 00:54:26] [INFO] [retrieval.status] Searching for: adult sci fi anime in space
[2026-01-15 00:54:26] [INFO] [indexing.status] Embedding 1 chunks with sentence-transformers/all-MiniLM-L6-v2
{'anime_id': 2202, 'title': 'Waga Seishun no Arcadia', 'score': 0.5656603254994922}
{'anime_id': 1652, 'title': 'Uchuu Senkan Yamato III', 'score': 0.5614717847745752}
{'anime_id': 1, 'title': 'Cowboy Bebop', 'score': 0.5573886281957738}
{'anime_id': 2451, 'title': 'Space Cobra', 'score': 0.5560254643591338}
{'anime_id': 5, 'title': 'Cowboy Bebop: Tengoku no Tobira', 'score': 0.5497831993896459}
{'anime_id': 1651, 'title': 'Uchuu Senkan Yamato 2', 'score': 0.5480509407395251}
{'anime_id': 1986, 'title': 'Wakusei Robo Danguard Ace', 'score': 0.5463270375681377}
{'anime_id': 184, 'title': 'Xenosaga The Animation', 'score': 0.5441384777460704}
{'anime_id': 474, 'title': 'Macross Plus', 'score': 0.5439777153904511}
{'anime_id': 2203, 'title': 'Waga Seishun no Arcadia: Mugen Kidou SSX', 'score

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from langchain_groq import ChatGroq
chat_model = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.1)

In [36]:
prompt = f"User wants anime recommendations for: `{query}`\n\nRank these anime:\n"
for i, c in enumerate(results):
    prompt += f"{i+1}. {c['title']}\n"
prompt += "\nReturn the best ones in order."

In [37]:
print(prompt)

User wants anime recommendations for: `adult sci fi anime in space`

Rank these anime:
1. Waga Seishun no Arcadia
2. Uchuu Senkan Yamato III
3. Cowboy Bebop
4. Space Cobra
5. Cowboy Bebop: Tengoku no Tobira
6. Uchuu Senkan Yamato 2
7. Wakusei Robo Danguard Ace
8. Xenosaga The Animation
9. Macross Plus
10. Waga Seishun no Arcadia: Mugen Kidou SSX

Return the best ones in order.


In [38]:
response = chat_model.invoke(prompt)

In [39]:
text = response.content 

In [41]:
ranked_titles = [
        line.split(". ", 1)[1]  
        for line in text.splitlines()
        if ". " in line
    ]

In [42]:
ranked_titles

["**Cowboy Bebop**: A classic anime that combines space exploration, jazz music, and a mix of action, drama, and comedy. It's a highly acclaimed and influential series that explores mature themes.",
 "**Macross Plus**: A sci-fi anime set in space, known for its mecha action, space battles, and a blend of music and drama. It's a well-regarded series that explores complex themes and has a strong focus on character development.",
 '**Waga Seishun no Arcadia: Mugen Kidou SSX**: This anime is a sequel to Waga Seishun no Arcadia and offers a more mature take on the sci-fi genre. It explores complex themes and has a strong focus on character development.',
 "**Xenosaga The Animation**: A sci-fi anime set in space, known for its complex storyline, deep characters, and philosophical themes. It's a well-regarded series that explores mature topics and has a strong focus on character development.",
 "**Uchuu Senkan Yamato 2**: A classic anime that combines space exploration, mecha action, and a mi

In [43]:
from langchain.agents import create_agent
from pydantic import BaseModel, Field

In [45]:
from typing import List
class RankedAnime(BaseModel):
    titles: List[str] = Field(description="List of anime titles sorted by rank")

In [48]:
agent = create_agent(model="gpt-4.1-mini", response_format=RankedAnime)

In [50]:
response = agent.invoke({"messages": prompt})

In [52]:
response["structured_response"].titles

['Cowboy Bebop',
 'Macross Plus',
 'Space Cobra',
 'Xenosaga The Animation',
 'Waga Seishun no Arcadia',
 'Waga Seishun no Arcadia: Mugen Kidou SSX',
 'Cowboy Bebop: Tengoku no Tobira',
 'Uchuu Senkan Yamato III',
 'Uchuu Senkan Yamato 2',
 'Wakusei Robo Danguard Ace']

In [3]:
from retrieval.search import search

In [21]:
filters = {
    "min_year": 2000,
    # "include_genres": ["Sci-Fi"],
    # "include_themes": ["Space"],
    # "min_score": 8,
}


results = search("adult space anime")


[2026-01-15 03:16:07] [INFO] [retrieval.status] Searching for: adult space anime
[2026-01-15 03:16:07] [INFO] [indexing.status] Embedding 1 chunks with sentence-transformers/all-MiniLM-L6-v2


In [22]:
print(*results, sep="\n")

{'anime_id': 1, 'title': 'Cowboy Bebop', 'score': 1.1213693256367114}
{'anime_id': 820, 'title': 'Ginga Eiyuu Densetsu', 'score': 1.10618474548923}
{'anime_id': 474, 'title': 'Macross Plus', 'score': 1.128518459947845}
{'anime_id': 5, 'title': 'Cowboy Bebop: Tengoku no Tobira', 'score': 1.1266259394009097}
{'anime_id': 1652, 'title': 'Uchuu Senkan Yamato III', 'score': 1.1341521544517508}
{'anime_id': 1651, 'title': 'Uchuu Senkan Yamato 2', 'score': 1.1218613122815588}
{'anime_id': 711, 'title': 'Uchuu Senkan Yamato (Movie)', 'score': 1.115980932724789}
{'anime_id': 2202, 'title': 'Waga Seishun no Arcadia', 'score': 1.1653352239375292}
{'anime_id': 2452, 'title': 'Space Adventure Cobra', 'score': 1.1306055891211726}
{'anime_id': 184, 'title': 'Xenosaga The Animation', 'score': 1.33130271860718}
{'anime_id': 2451, 'title': 'Space Cobra', 'score': 1.1442239787765436}
{'anime_id': 1662, 'title': 'Space Fantasia 2001 Nights', 'score': 1.0420486101819424}
{'anime_id': 2203, 'title': 'Waga S